Setup tensorflow object detection dependencies

Use tensorflow 1.0 since tensorflow 2.0 currently has issues converting to tflite

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf1/setup.py .
python -m pip install .

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
!pip3 install tf_slim

Test to make sure all of the dependencies were installed correctly.

In [ ]:

%cd /content/models/research/object_detection/builders/
!python model_builder_test.py


 Download the dataset of annotated images

In [ ]:
%cd /content/

In [ ]:
!wget http://cvrr.ucsd.edu/LISA/Datasets/signDatabasePublicFramesOnly.zip

In [ ]:
!unzip /content/signDatabasePublicFramesOnly.zip 

Generate a tfrecord from the annotated_images


In [ ]:
import csv
import requests
import sys
import getopt
from getopt import GetoptError
import traceback
import re
import os
import json
from collections import namedtuple
import requests
import tqdm
import tensorflow as tf
import random
from PIL import Image
from object_detection.utils import dataset_util

INPUT_FILE = 'allAnnotations.csv'
OUTPUT_FILE = 'traffic_signs.tfrecord'

def get_num_records(input_file):
  num_records = 0
  with open(input_file, 'rt') as csv_file:
      csv_reader = csv.DictReader(csv_file, delimiter=',')
      num_records = len(csv_file.readlines()) - 1
  return num_records

def create_tfrecord_from_csv(input_file, output_file, split = 0.9):
    writer = tf.io.TFRecordWriter(output_file)
    validation_record_file = re.sub(
      "\.tfrecord",
      "_val.tfrecord",
      output_file
    )
    validation_writter = tf.io.TFRecordWriter(validation_record_file)
    with open(input_file, 'rt') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=';')
        num_records = get_num_records(input_file)
        validation_index = int(num_records * split)
        count = 0
        file_dict = {}
        csv_data = []
        labels = []
        for row in tqdm.tqdm(csv_reader, total=num_records):
            if file_dict.get(row["Filename"]) is None:
                file_dict[row["Filename"]] = {
                    "label_names": [row["Annotation tag"]],
                    "x_minimums": [int(row["Upper left corner X"])],
                    "y_minimums":[int(row["Upper left corner Y"])],
                    "x_maximums":[int(row["Lower right corner X"])],
                    "y_maximums":[int(row["Lower right corner Y"])]
                }
            else:
                file_dict[row["Filename"]]["label_names"].append(row["Annotation tag"])
                file_dict[row["Filename"]]["x_minimums"].append(int(row["Upper left corner X"]))
                file_dict[row["Filename"]]["y_minimums"].append(int(row["Upper left corner Y"]))
                file_dict[row["Filename"]]["x_maximums"].append(int(row["Lower right corner X"]))
                file_dict[row["Filename"]]["y_maximums"].append(int(row["Lower right corner Y"]))
            
            if row["Annotation tag"] not in labels:
                labels.append(row["Annotation tag"])

        for row in tqdm.tqdm(file_dict):
            file_name = row.format(count=count).encode('utf8')
            with tf.io.gfile.GFile(file_name, 'rb') as f:
                image_data = f.read()

            image = Image.open(file_name)
            width, height = image.size
            for i in range(0, len(file_dict[row]['x_minimums'])):
                file_dict[row]['x_minimums'][i] = float(file_dict[row]['x_minimums'][i]) / float(width)
                file_dict[row]['x_maximums'][i] = float(file_dict[row]['x_maximums'][i]) / float(width)
                file_dict[row]['y_minimums'][i] = float(file_dict[row]['y_minimums'][i]) / float(height)
                file_dict[row]['y_maximums'][i] = float(file_dict[row]['y_maximums'][i]) / float(height)

            class_indexes = [] 
            for i in range(0, len(file_dict[row]['label_names'])):
                label = file_dict[row]['label_names'][i]
                class_indexes.append(labels.index(label) + 1)
                file_dict[row]['label_names'][i] = file_dict[row]['label_names'][i].encode('utf8')

            tf_example = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(int(height)),
                'image/width': dataset_util.int64_feature(int(width)),
                'image/filename': dataset_util.bytes_feature(file_name),
                'image/source_id': dataset_util.bytes_feature(file_name),
                'image/encoded': dataset_util.bytes_feature(image_data),
                'image/format': dataset_util.bytes_feature(b'png'),
                'image/object/bbox/xmin': dataset_util.float_list_feature(file_dict[row]['x_minimums']),
                'image/object/bbox/xmax': dataset_util.float_list_feature(file_dict[row]['x_maximums']),
                'image/object/bbox/ymin': dataset_util.float_list_feature(file_dict[row]['y_minimums']),
                'image/object/bbox/ymax': dataset_util.float_list_feature(file_dict[row]['y_maximums']),
                'image/object/class/text': dataset_util.bytes_list_feature(file_dict[row]["label_names"]),
                'image/object/class/label': dataset_util.int64_list_feature(class_indexes),
            }))
            
            if(count >= validation_index):
                validation_writter.write(tf_example.SerializeToString())
            else:
                writer.write(tf_example.SerializeToString())
            count += 1
        create_pbtxt_file(labels)
        writer.close()
        validation_writter.close()
 
def create_pbtxt_file(labels):
  with open('label_map.pbtxt', 'wt') as label_file:
    i = 1
    for label in labels:
      item_str = "\tid: {id}\n\tname: '{name}'\n".format(id=i, name=label)
      item_str = "item{\n" + item_str + "}"
      if (i != len(labels)):
        item_str += ",\n"
      else:
        item_str += "\n"
      label_file.write(
          item_str
      )
      i +=1

create_tfrecord_from_csv(INPUT_FILE, OUTPUT_FILE)


In [ ]:

test_record_fname = '/content/traffic_signs_val.tfrecord'
train_record_fname = '/content/traffic_signs.tfrecord'
label_map_pbtxt_fname = '/content/label_map.pbtxt'

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=100, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
#download base training configuration file
model_name = 'ssd_mobilenet_v3_small_coco_2020_01_14'
download_config = 'http://download.tensorflow.org/models/object_detection/{model_name}.tar.gz'.format(model_name=model_name)
!wget {download_config}
!tar -zxvf { model_name + '.tar.gz'}

In [ ]:
#prepare
pipeline_fname = '/content/' + model_name + '/pipeline.config'
fine_tune_checkpoint = '/content/' + model_name + '/model.ckpt'
batch_size = 64
num_steps = 2000

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd {'/content/' + model_name + '/'}
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        'input_path: "PATH_TO_BE_CONFIGURED\/.*_train.*"', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        'input_path: "PATH_TO_BE_CONFIGURED\/.*_val.*"', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
     #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    
    s = re.sub(
        'learning_rate_base: \d\.\d*', 'learning_rate_base: {}'.format(0.001), s)
    s = re.sub(
        'warmup_learning_rate: \d\.\d*', 'warmup_learning_rate: {}'.format(0.001), s)
    f.write(s)



In [ ]:
%cat {pipeline_fname}

In [ ]:
!cp /content/models/research/object_detection/legacy/train.py /content/train.py
%cd /content/
!python /content/train.py --logtostderr --train_dir=training/ --pipeline_config_path={pipeline_fname}

Exporting the model

In [ ]:
%cd /content/models/research/object_detection
!python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path {pipeline_fname} --trained_checkpoint_prefix /content/training/model.ckpt-2000 --output_directory /content/trained_model

Testing the model


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = '/content/trained_model'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/label_map.pbtxt'



### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)



def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
    
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


image_path = '/content/vid0/stop_1323804419.avi_image32.png'
image = Image.open(image_path)
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)
# Actual detection.
output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
print(output_dict)
# Visualization of the results of a detection.
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks'),
    use_normalized_coordinates=True,
    line_thickness=1)
display(Image.fromarray(image_np))

Convert to tflite

In [ ]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/trained_model/pipeline.config \
--trained_checkpoint_prefix=/content/trained_model/model.ckpt \
--output_directory=/content/tflite \
--add_postprocessing_op=true

In [ ]:
!sudo apt install curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!sudo mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel
!sudo apt install bazel-3.1.0

In [ ]:
%cd /content/
!git clone --recurse-submodules https://github.com/tensorflow/tensorflow.git

In [ ]:
%cd /content/tensorflow/
!bazel run -c opt tensorflow/lite/toco:toco -- \
--input_file=/content/tflite/tflite_graph.pb \
--output_file=/content/tflite/detect.tflite \
--input_shapes=1,320,320,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=FLOAT \
--mean_values=128 \
--std_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops